# Case Study : Leads and Purchase - Many to many join
1. Leads data
2. Purchase data - Purchase should be within 180 days of lead creation

## pandas.merge - arguments
- left
- right
- how : Default inner - left, right, inner, outer, cross
- on : Common key
- left_on :  Key from left dataframe
- right_on : Key from right dataframe
- left_index : Dafault False - uses index from the left dataframe to join
- right_index : Default False
- sort : Default False, sorts on the key
- suffixes : Default ("_x", "_y")
- copy : default True
- indicator : default False : adds a column "_merge" to the dataframe with merge information
- validate : optional - checks if merge is 1:m, m:1, 1:1

In [1]:
import pandas as pd


## Reading the files

In [2]:
leads = pd.read_excel("Leads_Purchase.xlsx", sheet_name= 0)

leads.head()

,LeadId,CustomerID,Product,Date,Year_Month
0,1001,3047,A,2013-09-10,2013_09
1,1002,4782,A,2015-11-19,2015_11
2,1003,6896,D,2019-08-09,2019_08
3,1004,3014,C,2014-11-28,2014_11
4,1005,1409,E,2015-07-02,2015_07


In [3]:
# Convert the 'Date' column to datetime format and then format it to 'yyyy-mm-dd'
leads['Date'] = pd.to_datetime(leads['Date']).dt.strftime('%Y-%m-%d')

leads.head()

,LeadId,CustomerID,Product,Date,Year_Month
0,1001,3047,A,2013-09-10,2013_09
1,1002,4782,A,2015-11-19,2015_11
2,1003,6896,D,2019-08-09,2019_08
3,1004,3014,C,2014-11-28,2014_11
4,1005,1409,E,2015-07-02,2015_07


In [4]:
leads.dtypes

LeadId         int64
CustomerID     int64
Product       object
Date          object
Year_Month    object
dtype: object

In [5]:
leads['CustomerID'].value_counts()

CustomerID
1306    3
1204    3
891     3
1472    3
3767    3
       ..
1665    1
2553    1
480     1
469     1
1525    1
Name: count, Length: 100, dtype: int64

In [6]:
leads.groupby(['CustomerID', 'Product', 'Date']).size()

CustomerID  Product  Date      
13          A        2014-03-27    1
72          A        2013-07-18    1
                     2016-08-11    1
            D        2013-01-20    1
187         D        2014-03-20    1
                                  ..
9654        C        2017-07-17    1
9707        D        2022-06-07    1
9896        C        2020-05-20    1
            E        2017-05-13    1
9994        A        2014-11-06    1
Length: 181, dtype: int64

In [7]:
purchase = pd.read_excel("Leads_Purchase.xlsx", sheet_name= 1)

purchase.head()

,PurchaseID,CustomerID,Product,Purchase_Date
0,2001,3047,A,2013-12-16
1,2002,6896,D,2019-10-29
2,2003,1409,E,2015-08-01
3,2004,187,D,2014-07-06
4,2005,5592,B,2020-03-31


In [8]:
purchase['Purchase_Date'] = pd.to_datetime(purchase['Purchase_Date']).dt.strftime('%Y-%m-%d')

In [9]:
purchase.head()

,PurchaseID,CustomerID,Product,Purchase_Date
0,2001,3047,A,2013-12-16
1,2002,6896,D,2019-10-29
2,2003,1409,E,2015-08-01
3,2004,187,D,2014-07-06
4,2005,5592,B,2020-03-31


In [10]:
purchase.groupby(['CustomerID', 'Product', 'Purchase_Date']).size()

CustomerID  Product  Purchase_Date
13          A        2014-05-24       1
72          A        2013-09-14       1
                     2016-09-09       1
187         D        2014-07-06       1
380         B        2013-07-26       1
                                     ..
8738        E        2024-09-02       1
8957        C        2022-11-30       1
9138        B        2022-03-16       1
9185        C        2023-02-21       1
9707        D        2022-10-16       1
Length: 87, dtype: int64

In [11]:
purchase['CustomerID'].value_counts()

CustomerID
1306    3
1204    3
5291    2
5814    2
1472    2
       ..
5476    1
1665    1
480     1
7760    1
4631    1
Name: count, Length: 66, dtype: int64

## Question :  Assign Leads to the correct purchase based on closest lead date and purchase date

In [12]:
print('Leads shape: ', leads.shape)
print('Purchase shape: ', purchase.shape)

Leads shape:  (181, 5)
Purchase shape:  (87, 4)


### Three leads for customer id 72

In [13]:
leads[leads['CustomerID'] == 72]

,LeadId,CustomerID,Product,Date,Year_Month
47,1048,72,D,2013-01-20,2013_01
147,1148,72,A,2013-07-18,2013_07
178,1179,72,A,2016-08-11,2016_08


### Two purchase for customer id 72

In [14]:
purchase[purchase['CustomerID'] == 72]

,PurchaseID,CustomerID,Product,Purchase_Date
75,2076,72,A,2013-09-14
86,2087,72,A,2016-09-09


### Merge leads and purchase table on both the columns - CustomerID and Product

In [15]:
leads_purchase2 = pd.merge(leads, purchase, 
                           left_on = ['CustomerID', 'Product'], 
                           right_on = ['CustomerID', 'Product'], 
                           how = 'left')

leads_purchase2.shape

(194, 7)

In [16]:
leads_purchase2[leads_purchase2['Purchase_Date'].notna()]

,LeadId,CustomerID,Product,Date,Year_Month,PurchaseID,Purchase_Date
0,1001,3047,A,2013-09-10,2013_09,2001.0,2013-12-16
2,1003,6896,D,2019-08-09,2019_08,2002.0,2019-10-29
4,1005,1409,E,2015-07-02,2015_07,2003.0,2015-08-01
10,1011,187,D,2014-03-20,2014_03,2004.0,2014-07-06
12,1013,5592,B,2019-10-05,2019_10,2005.0,2020-03-31
...,...,...,...,...,...,...,...
186,1175,4950,C,2023-05-21,2023_05,2074.0,2019-02-27
188,1177,4631,A,2019-05-05,2019_05,2085.0,2019-06-18
189,1178,8686,E,2015-02-07,2015_02,2086.0,2015-07-18
190,1179,72,A,2016-08-11,2016_08,2076.0,2013-09-14


### Total five leads - purchase combination for customer id 72
- There is no purchase for customer id 72 and Product D combination
- There are two purchase and two leads for customer id 72 and Product A combinations i.e total 4 records after joins

In [17]:
leads_purchase2[leads_purchase2['CustomerID'] == 72]

,LeadId,CustomerID,Product,Date,Year_Month,PurchaseID,Purchase_Date
51,1048,72,D,2013-01-20,2013_01,NaN,NaN
154,1148,72,A,2013-07-18,2013_07,2076.0,2013-09-14
155,1148,72,A,2013-07-18,2013_07,2087.0,2016-09-09
190,1179,72,A,2016-08-11,2016_08,2076.0,2013-09-14
191,1179,72,A,2016-08-11,2016_08,2087.0,2016-09-09


In [18]:
leads_purchase2['Days_2_Purchase'] = (pd.to_datetime(leads_purchase2['Purchase_Date']) - 
pd.to_datetime(leads_purchase2['Date'])).dt.days

In [19]:
leads_purchase2

,LeadId,CustomerID,Product,Date,Year_Month,PurchaseID,Purchase_Date,Days_2_Purchase
0,1001,3047,A,2013-09-10,2013_09,2001.0,2013-12-16,97.0
1,1002,4782,A,2015-11-19,2015_11,NaN,NaN,NaN
2,1003,6896,D,2019-08-09,2019_08,2002.0,2019-10-29,81.0
3,1004,3014,C,2014-11-28,2014_11,NaN,NaN,NaN
4,1005,1409,E,2015-07-02,2015_07,2003.0,2015-08-01,30.0
...,...,...,...,...,...,...,...,...
189,1178,8686,E,2015-02-07,2015_02,2086.0,2015-07-18,161.0
190,1179,72,A,2016-08-11,2016_08,2076.0,2013-09-14,-1062.0
191,1179,72,A,2016-08-11,2016_08,2087.0,2016-09-09,29.0
192,1180,8012,C,2017-03-08,2017_03,NaN,NaN,NaN


In [20]:
leads_purchase2.sort_values(by = ['CustomerID', 'Product', 'Date', 'Days_2_Purchase'], 
                           ascending = [True, True, True, True], inplace = True)

In [21]:
leads_purchase2

,LeadId,CustomerID,Product,Date,Year_Month,PurchaseID,Purchase_Date,Days_2_Purchase
98,1095,13,A,2014-03-27,2014_03,2046.0,2014-05-24,58.0
154,1148,72,A,2013-07-18,2013_07,2076.0,2013-09-14,58.0
155,1148,72,A,2013-07-18,2013_07,2087.0,2016-09-09,1149.0
190,1179,72,A,2016-08-11,2016_08,2076.0,2013-09-14,-1062.0
191,1179,72,A,2016-08-11,2016_08,2087.0,2016-09-09,29.0
...,...,...,...,...,...,...,...,...
100,1097,9654,C,2017-07-17,2017_07,NaN,NaN,NaN
84,1081,9707,D,2022-06-07,2022_06,2039.0,2022-10-16,131.0
16,1017,9896,C,2020-05-20,2020_05,NaN,NaN,NaN
120,1117,9896,E,2017-05-13,2017_05,NaN,NaN,NaN


In [22]:
leads_purchase2[leads_purchase2['CustomerID'] == 72]

,LeadId,CustomerID,Product,Date,Year_Month,PurchaseID,Purchase_Date,Days_2_Purchase
154,1148,72,A,2013-07-18,2013_07,2076.0,2013-09-14,58.0
155,1148,72,A,2013-07-18,2013_07,2087.0,2016-09-09,1149.0
190,1179,72,A,2016-08-11,2016_08,2076.0,2013-09-14,-1062.0
191,1179,72,A,2016-08-11,2016_08,2087.0,2016-09-09,29.0
51,1048,72,D,2013-01-20,2013_01,NaN,NaN,NaN


In [23]:
leads_purchase2 = leads_purchase2[(leads_purchase2['Days_2_Purchase'] >= 0)&(leads_purchase2['Days_2_Purchase'] <= 180)]

In [24]:
leads_purchase_one = leads_purchase2.groupby(['CustomerID', 'Product', 'Date']).head(1)

In [25]:
leads_purchase_one

,LeadId,CustomerID,Product,Date,Year_Month,PurchaseID,Purchase_Date,Days_2_Purchase
98,1095,13,A,2014-03-27,2014_03,2046.0,2014-05-24,58.0
154,1148,72,A,2013-07-18,2013_07,2076.0,2013-09-14,58.0
191,1179,72,A,2016-08-11,2016_08,2087.0,2016-09-09,29.0
10,1011,187,D,2014-03-20,2014_03,2004.0,2014-07-06,108.0
65,1062,380,B,2013-02-09,2013_02,2029.0,2013-07-26,167.0
...,...,...,...,...,...,...,...,...
117,1114,8738,E,2024-04-27,2024_04,2056.0,2024-09-02,128.0
71,1068,8957,C,2022-10-15,2022_10,2034.0,2022-11-30,46.0
102,1099,9138,B,2022-02-28,2022_02,2049.0,2022-03-16,16.0
139,1135,9185,C,2023-01-25,2023_01,2067.0,2023-02-21,27.0


In [26]:
leads_purchase_one.groupby(['CustomerID', 'Product', 'Date']).size()

CustomerID  Product  Date      
13          A        2014-03-27    1
72          A        2013-07-18    1
                     2016-08-11    1
187         D        2014-03-20    1
380         B        2013-02-09    1
                                  ..
8738        E        2024-04-27    1
8957        C        2022-10-15    1
9138        B        2022-02-28    1
9185        C        2023-01-25    1
9707        D        2022-06-07    1
Length: 87, dtype: int64

## Get conversion %
It is defined as number of converted leads who made purchase/ total leads

### Get the leads not present in mapped leads_purchase data

In [27]:
leads_only = pd.merge(leads, leads_purchase_one[['LeadId']],
                     on = ['LeadId'],
                     how = 'left',
                     suffixes = ("", "_"),
                     indicator = True)



In [28]:
leads_only['_merge'].value_counts()

_merge
left_only     94
both          87
right_only     0
Name: count, dtype: int64

In [29]:
leads_only = leads_only[leads_only['_merge'] == 'left_only']

In [30]:
leads_only

,LeadId,CustomerID,Product,Date,Year_Month,_merge
1,1002,4782,A,2015-11-19,2015_11,left_only
3,1004,3014,C,2014-11-28,2014_11,left_only
5,1006,6932,C,2016-06-11,2016_06,left_only
6,1007,6258,C,2022-06-10,2022_06,left_only
7,1008,8034,D,2014-11-13,2014_11,left_only
...,...,...,...,...,...,...
173,1174,6855,B,2014-01-12,2014_01,left_only
174,1175,4950,C,2023-05-21,2023_05,left_only
175,1176,6938,B,2019-01-18,2019_01,left_only
179,1180,8012,C,2017-03-08,2017_03,left_only


In [31]:
all_leads = pd.concat([leads_only, leads_purchase_one])

all_leads

,LeadId,CustomerID,Product,Date,Year_Month,_merge,PurchaseID,Purchase_Date,Days_2_Purchase
1,1002,4782,A,2015-11-19,2015_11,left_only,NaN,NaN,NaN
3,1004,3014,C,2014-11-28,2014_11,left_only,NaN,NaN,NaN
5,1006,6932,C,2016-06-11,2016_06,left_only,NaN,NaN,NaN
6,1007,6258,C,2022-06-10,2022_06,left_only,NaN,NaN,NaN
7,1008,8034,D,2014-11-13,2014_11,left_only,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
117,1114,8738,E,2024-04-27,2024_04,NaN,2056.0,2024-09-02,128.0
71,1068,8957,C,2022-10-15,2022_10,NaN,2034.0,2022-11-30,46.0
102,1099,9138,B,2022-02-28,2022_02,NaN,2049.0,2022-03-16,16.0
139,1135,9185,C,2023-01-25,2023_01,NaN,2067.0,2023-02-21,27.0


In [32]:
all_leads['Converted'] = all_leads['PurchaseID'].apply(lambda x: 1 if pd.notna(x) else 0)

In [33]:
all_leads['Converted'].value_counts()

Converted
0    94
1    87
Name: count, dtype: int64

In [36]:
all_leads['Converted'].value_counts(normalize = True).round(2)

Converted
0    0.52
1    0.48
Name: proportion, dtype: float64

### Get conversion % by Product

In [35]:
all_leads.groupby(['Product']).agg(leads = ('LeadId', 'count'),
                                   Converted = ('Converted', 'sum'),
                                   Conversion_Perc = ('Converted', 'mean')).round(2)

,leads,Converted,Conversion_Perc
Product,,,
A,45,24,0.53
B,28,14,0.50
C,42,14,0.33
D,35,19,0.54
E,31,16,0.52
